<a href="https://colab.research.google.com/github/bennyqp/artificial-inspiration/blob/main/Image%20Generation/ai_image_generation_deepdream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepDreaming with TensorFlow
_
###Deep Dream Code and original Notebook from
## ___Alex Mordvintsev___
https://colab.research.google.com/drive/1DWcrN9WXni58MbddvlShX0wF_oeo8W_0

_

This notebook was modified for batch processing as part of the "Artificial Inspiration" project. 

Here is the link to the project:

https://github.com/bennyqp/artificial-inspiration

#Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1) Load the model graph

The pretrained Inception network can be downloaded [here](https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip). This next cell downloads the file automatically and unpacks it locally to the Colab kernel. We can the load the contained model file  'tensorflow_inception_graph.pb' in the cell below.

In [ ]:
!wget -nc --no-check-certificate https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip && unzip -n inception5h.zip

from io import BytesIO
from IPython.display import clear_output, Image, display
import numpy as np
import PIL.Image
import tensorflow.compat.v1 as tf
from __future__ import print_function
import os

model_fn = 'tensorflow_inception_graph.pb'

# creating TensorFlow session and loading the model
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
with tf.gfile.FastGFile(model_fn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
t_input = tf.placeholder(np.float32, name='input') # define the input tensor
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean, 0)
tf.import_graph_def(graph_def, {'input':t_preprocessed})

def T(layer):
    '''Helper for getting layer output tensor'''
    return graph.get_tensor_by_name("import/%s:0"%layer)

### 2) The core deepdream code

In [ ]:
# These parameters let us control the strenth of the deepdream.
octave_n = 4
octave_scale = 1.4
iter_n = 10
strength = 200

# Helper function that uses TensorFlow to resize an image
def resize(img, new_size):
    return sess.run(tf.image.resize_bilinear(img[np.newaxis,:], new_size))[0]

# Apply gradients to an image in a seires of tiles
def calc_grad_tiled(img, t_grad, tile_size=256):
    '''Random shifts are applied to the image to blur tile boundaries over
    multiple iterations.'''
    h, w = img.shape[:2]
    sx, sy = np.random.randint(tile_size, size=2)
    # We randomly roll the image in x and y to avoid seams between tiles.
    img_shift = np.roll(np.roll(img, sx, 1), sy, 0)
    grad = np.zeros_like(img)
    for y in range(0, max(h-tile_size//2, tile_size),tile_size):
        for x in range(0, max(w-tile_size//2, tile_size),tile_size):
            sub = img_shift[y:y+tile_size,x:x+tile_size]
            g = sess.run(t_grad, {t_input:sub})
            grad[y:y+tile_size,x:x+tile_size] = g
    imggrad = np.roll(np.roll(grad, -sx, 1), -sy, 0)
    # Add the image gradient to the image and return the result
    return img + imggrad*(strength * 0.01 / (np.abs(imggrad).mean()+1e-7))

# Applies deepdream at multiple scales
def render_deepdream(t_obj, input_img, show_steps = True):
    # Collapse the optimization objective to a single number (the loss)
    t_score = tf.reduce_mean(t_obj)
    # We need the gradient of the image with respect to the objective
    t_grad = tf.gradients(t_score, t_input)[0]

    # split the image into a number of octaves (laplacian pyramid)
    img = input_img
    octaves = []
    for i in range(octave_n-1):
        lo = resize(img, np.int32(np.float32(img.shape[:2])/octave_scale))
        octaves.append(img-resize(lo, img.shape[:2]))
        img = lo

    # generate details octave by octave
    for octave in range(octave_n):
        if octave>0:
            hi = octaves[-octave]
            img = resize(img, hi.shape[:2])+hi
        for i in range(iter_n):
            img = calc_grad_tiled(img, t_grad)
        if show_steps:
            clear_output()
            showarray(img)
    return img

# Display DeepDream Image
def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

### 3) Randomness

For batch processing, we want to use randomly chosen parameters to always get new results. Here we define the functions for this. 

You can play around with the parameters to get different results. 

In [ ]:
import random

layers = ["mixed3a", 
          "mixed3b", 
          "mixed4a", 
          "mixed4c", 
          "mixed5a"]

def random_octave():
  return random.randint(1, 10)

def random_octave_scale():
  return random.uniform(1.0, 1.9)

def random_iter_n():
  return random.randint(1, 50)

def random_strength():
  return random.randint(1, 1000)

def random_layer():
  layerNum = random.randint(0, len(layers)-1)
  return(layers[layerNum])

### Now let's start generating more and more Deep Dream images

First, we define an input path. This should lead to a folder with one or more. 
We also need an output path: a folder where the created images will be saved. 


In [ ]:
#Input and Output Paths 
input_path = "/content/input"
output_path = "/content/output"

We can also specify how many Deep Dream variants should be created per image. 

All files from the input folder are then processed and a corresponding number of image variations are generated for each image. 

We can also specify at which image in the folder we want to start. 

In [ ]:
image_variations = 3
startWithImage = 0

Now let's start generating images! 

In [ ]:
dirs = os.listdir( input_path )
dirs = dirs[(startWithImage-1):]

for image in dirs:
  if not image.startswith('.'):
    for i in range(image_variations):
      file_contents = open(os.path.join(input_path, image)).read()  
      #print(file_contents)
      img0 = sess.run(tf.image.decode_image(file_contents))

      octave_n = random_octave() 
      octave_scale =  random_octave_scale()
      iter_n = random_iter_n() 
      strength = random_strength() 
      layer = random_layer()

      final = render_deepdream(tf.square(T(layer)), img0)

      final = np.uint8(np.clip(final, 0, 255))
      PIL.Image.fromarray(final).save(os.path.join(output_path, os.path.splitext(image)[0] + "_dd0" + str(i) + ".png"))
print("done.")

Do you need to move the output folder somewhere?

In [ ]:
import shutil

original = output_path
target = r'/content/drive/My Drive/Bachelorarbeit/testData'

shutil.move(original,target)
os.mkdir(original)